In [20]:
from utils import process_tweet, lookup
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd
import w2_unittest
nltk.download('twitter_sampse')
nltk.download('stopwords')

[nltk_data] Error loading twitter_sampse: Package 'twitter_sampse' not
[nltk_data]     found in index
[nltk_data] Downloading package stopwords to /home/sadra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
filePath = f'{getcwd()}/../tmp2'
nltk.data.path.append(filePath)

In [22]:
# get the sets of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

## split the data insto tow pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

## avoind assumption about the length of all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y =  np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))
print(test_y, test_y.shape)

[1. 1. 1. ... 0. 0. 0.] (2000,)


In [23]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"
print(process_tweet(custom_tweet))

['hello', 'great', 'day', ':)', 'good', 'morn']


In [24]:
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    for y, tweet in zip(ys,tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            ## if the key exist in the dictionary, increment the count
            if pair in result:
                result[pair]+=1
            ## else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1
    return result

In [25]:
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1,0,0,0,0]
count_tweets(result,tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

In [26]:
## Build the freqs dictionary for later uses
freqs = count_tweets({}, train_x, train_y)

In [27]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0
    
    # calculatie the v, the number of unique words in the vocabulary
    vocab = set(pair[0] for pair in freqs.keys())
    V = len(vocab)
    
    ## calculate N_pos and N_neg
    
    N_pos = N_neg = 0
    for pair in freqs.keys():
        if pair[1] > 0:
            N_pos =+ freqs[pair]
        else:
            N_neg =+ freqs[pair]
        
    ## calculate D, the number of document
    D = len(train_x)
    
    ## calcualte D_pos, the number of positive tweets
    D_pos = np.count_nonzero(train_y==1)
    
    ## calculate D_neg
    D_neg = np.count_nonzero(train_y==0)
    
    ## calculate the logprior
    logprior = np.log(D_pos/D_neg)
    
    ### for each word in the vocabulary 
    for word in vocab:
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word,0)
        
        ## calculate the probability that each word is pos or neg
        p_w_pos = (freq_pos + 1)/(N_pos + V)
        p_w_neg = (freq_neg + 1)/(N_neg + V)
        
        ## calcualte the loglikelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
    
    return logprior, loglikelihood
    
            

In [28]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9162


In [29]:
## naive bayes predict

def naive_bayes_predict(tweet,logprior, loglikelihood):
   
    ## process the tweet
    word_l = process_tweet(tweet)
     ## initialize probability to zero
        
    p = 0
    
    ## add the logprior
    p += logprior
    
    for word in word_l:
        if word in loglikelihood:
            ## add the log likelihood of that word to the probability
            p += loglikelihood[word]
            
    return p

In [31]:
my_tweet = 'she smiled'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('the expected output is: ', p)

the expected output is:  1.5686159179138452


In [32]:
## experiment with your own tweet
my_tweet = 'He failed the exam'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

-0.4700036292457356


In [36]:
## test naive bayes
def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=naive_bayes_predict):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0
    y_hats = []
    for tweet in test_x:
        ## if the prediction is > 0
        if naive_bayes_predict(tweet,logprior,loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        
        # append the prdicted class to the list of y_hats
        y_hats.append(y_hat_i)
        
    ## error is the average of the absolute values of the differnces between y_hats and test_y
    error = np.mean([np.abs(y_hats[i] - test_y[i]) for i in range(len(y_hats))])
    accuracy = 1 - error
    
    return accuracy

In [37]:
print('Naive Bayes accuracy = %.4f'%(test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9955


In [38]:
# UNQ_C7 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.30
this movie should have been great. -> 2.14
great -> 2.14
great great -> 4.28
great great great -> 6.42
great great great great -> 8.56


In [39]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'you are bad :('
naive_bayes_predict(my_tweet, logprior, loglikelihood)

-8.815716287045893

In [44]:
# UNQ_C8 GRADED FUNCTION: get_ratio

def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    
    pos_neg_ratio = {'positive': 0, 'negative':0, 'ratio':0}
    
    # use lookup to find the positive counts for word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)
    
    ## use lookup to find negative counts for word (denoted by the integer 0)
    pos_neg_ratio['negative'] = lookup(freqs, word,0)
    
    ## calculate the ratio of positive to negative counts for the word
    
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive']+1)/(pos_neg_ratio['negative']+1)
    
    return pos_neg_ratio

In [45]:
get_ratio(freqs, 'happi')

{'positive': 162, 'negative': 18, 'ratio': 8.578947368421053}